In [ ]:
import pandas as pd
import numpy as np

file_path = '/content/dataset.csv'

# Load the dataset
df = pd.read_csv(file_path)

# Preview the first few rows
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [ ]:
# Create timestamp from date and time columns
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Calculate occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Convert traffic condition to numeric
df['TrafficLevel'] = df['TrafficConditionNearby'].map({
    'low': 1,
    'average': 2,
    'high': 3
})

# Map vehicle types to weights
vehicle_weights = {
    'car': 1.0,
    'bike': 0.7,
    'truck': 1.5,
    'cycle': 0.5
}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

# Display preview of new columns
df[['Timestamp', 'OccupancyRate', 'TrafficLevel', 'VehicleType', 'VehicleWeight']].head()


,Timestamp,OccupancyRate,TrafficLevel,VehicleType,VehicleWeight
0,2016-10-04 07:59:00,0.105719,1,car,1.0
1,2016-10-04 08:25:00,0.110919,1,car,1.0
2,2016-10-04 08:59:00,0.138648,1,car,1.0
3,2016-10-04 09:32:00,0.185442,1,car,1.0
4,2016-10-04 09:59:00,0.259965,1,bike,0.7


In [ ]:
# Create timestamp from date and time columns
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Calculate occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Convert traffic condition to numeric
df['TrafficLevel'] = df['TrafficConditionNearby'].map({
    'low': 1,
    'average': 2,
    'high': 3
})

# Map vehicle types to weights
vehicle_weights = {
    'car': 1.0,
    'bike': 0.7,
    'truck': 1.5,
    'cycle': 0.5
}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

# Display preview of new columns
df[['Timestamp', 'OccupancyRate', 'TrafficLevel', 'VehicleType', 'VehicleWeight']].head()


,Timestamp,OccupancyRate,TrafficLevel,VehicleType,VehicleWeight
0,2016-10-04 07:59:00,0.105719,1,car,1.0
1,2016-10-04 08:25:00,0.110919,1,car,1.0
2,2016-10-04 08:59:00,0.138648,1,car,1.0
3,2016-10-04 09:32:00,0.185442,1,car,1.0
4,2016-10-04 09:59:00,0.259965,1,bike,0.7


In [ ]:
# Model 1: Price increases linearly with occupancy rate
# Starting price is $10, and price increases based on occupancy

df['Price_Model1'] = 10 + 5 * df['OccupancyRate']

# Ensure price stays within realistic bounds ($5 to $20)
df['Price_Model1'] = df['Price_Model1'].clip(lower=5, upper=20)

# Preview the results
df[['Timestamp', 'SystemCodeNumber', 'OccupancyRate', 'Price_Model1']].head()


,Timestamp,SystemCodeNumber,OccupancyRate,Price_Model1
0,2016-10-04 07:59:00,BHMBCCMKT01,0.105719,10.528596
1,2016-10-04 08:25:00,BHMBCCMKT01,0.110919,10.554593
2,2016-10-04 08:59:00,BHMBCCMKT01,0.138648,10.693241
3,2016-10-04 09:32:00,BHMBCCMKT01,0.185442,10.927210
4,2016-10-04 09:59:00,BHMBCCMKT01,0.259965,11.299827


In [ ]:
# Define coefficients for demand function
alpha = 1.0    # occupancy rate
beta = 0.2     # queue length
gamma = 0.5    # traffic level
delta = 0.3    # special day indicator
epsilon = 0.5  # vehicle type weight

# Calculate raw demand score
df['RawDemand'] = (
    alpha * df['OccupancyRate'] +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalize demand to [0, 1]
d_min = df['RawDemand'].min()
d_max = df['RawDemand'].max()
df['NormalizedDemand'] = (df['RawDemand'] - d_min) / (d_max - d_min)

# Apply demand to adjust price (base price = $10)
lambda_param = 0.8
df['Price_Model2'] = 10 * (1 + lambda_param * df['NormalizedDemand'])

# Clip to stay within realistic bounds
df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20)

# Preview results
df[['Timestamp', 'OccupancyRate', 'QueueLength', 'TrafficLevel', 'Price_Model2']].head()


,Timestamp,OccupancyRate,QueueLength,TrafficLevel,Price_Model2
0,2016-10-04 07:59:00,0.105719,1,1,11.618829
1,2016-10-04 08:25:00,0.110919,1,1,11.630336
2,2016-10-04 08:59:00,0.138648,2,1,12.134338
3,2016-10-04 09:32:00,0.185442,2,1,12.237900
4,2016-10-04 09:59:00,0.259965,2,1,12.070859


In [ ]:
from math import radians, sin, cos, sqrt, atan2

# ✅ Step 1: Haversine formula to calculate distance between two lat-lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# ✅ Step 2: Get lat/lon of all parking lots
lot_coords = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first().to_dict('index')

# ✅ Step 3: Function to return nearby lots (within 2 km)
def get_nearby_lots(lot_id):
    lat1, lon1 = lot_coords[lot_id]["Latitude"], lot_coords[lot_id]["Longitude"]
    return [
        other_id for other_id in lot_coords
        if other_id != lot_id and
        haversine(lat1, lon1, lot_coords[other_id]["Latitude"], lot_coords[other_id]["Longitude"]) < 2
    ]

# ✅ Step 4: Test on one sample lot
sample_lot = df['SystemCodeNumber'].unique()[0]
print(f"Nearby lots for {sample_lot}:")
print(get_nearby_lots(sample_lot))



Nearby lots for BHMBCCMKT01:
['BHMBCCTHL01', 'BHMEURBRD01', 'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98', 'Shopping']


In [ ]:
# Initialize Model 3 pricing and suggested lot columns
df['Price_Model3'] = df['Price_Model2']
df['SuggestedLot'] = ""

# Loop over each lot and its data
for lot_id in df['SystemCodeNumber'].unique():
    nearby_lots = get_nearby_lots(lot_id)

    if not nearby_lots:
        continue  # skip if no nearby lots

    # Filter data for the current lot
    current_lot_data = df[df['SystemCodeNumber'] == lot_id]

    for idx, row in current_lot_data.iterrows():
        timestamp = row['Timestamp']

        # Find competitor lots at the same time
        competitors = df[
            (df['SystemCodeNumber'].isin(nearby_lots)) &
            (df['Timestamp'] == timestamp)
        ]

        # Apply rerouting logic only if the lot is full and competitors exist
        if row['Occupancy'] >= row['Capacity'] and not competitors.empty:
            # Find cheapest competitor at that timestamp
            cheapest = competitors.loc[competitors['Price_Model2'].idxmin()]

            # If competitor is cheaper, update price and suggest rerouting
            if cheapest['Price_Model2'] < row['Price_Model2']:
                df.at[idx, 'Price_Model3'] = cheapest['Price_Model2']
                df.at[idx, 'SuggestedLot'] = cheapest['SystemCodeNumber']

# Preview only the rows where rerouting was suggested
df[df['SuggestedLot'] != ""][['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity', 'Price_Model2', 'Price_Model3', 'SuggestedLot']].head()



,Timestamp,SystemCodeNumber,Occupancy,Capacity,Price_Model2,Price_Model3,SuggestedLot
2075,2016-11-17 11:31:00,BHMBCCTHL01,387,387,14.261964,12.570709,Others-CCCPS119a
2076,2016-11-17 12:04:00,BHMBCCTHL01,390,387,14.168463,12.297161,BHMBCCMKT01
2077,2016-11-17 12:31:00,BHMBCCTHL01,392,387,14.511874,12.796762,Others-CCCPS119a
2078,2016-11-17 13:04:00,BHMBCCTHL01,390,387,14.057805,12.205681,BHMBCCMKT01
2079,2016-11-17 13:31:00,BHMBCCTHL01,394,387,14.080680,11.792944,Others-CCCPS119a


from matplotlib import pyplot as plt
_df_0['Occupancy'].plot(kind='hist', bins=20, title='Occupancy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Price_Model2'].plot(kind='hist', bins=20, title='Price_Model2')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Price_Model3'].plot(kind='hist', bins=20, title='Price_Model3')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('SuggestedLot').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['SuggestedLot'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='Occupancy', y='SuggestedLot', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['SuggestedLot'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Price_Model2', y='SuggestedLot', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['SuggestedLot'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Price_Model3', y='SuggestedLot', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

# Select one parking lot to visualize
lot_id = df['SystemCodeNumber'].unique()[0]  # you can replace with a specific ID

lot_df = df[df['SystemCodeNumber'] == lot_id].sort_values("Timestamp")

# Create a data source for Bokeh
source = ColumnDataSource(data={
    'x': lot_df['Timestamp'],
    'price1': lot_df['Price_Model1'],
    'price2': lot_df['Price_Model2'],
    'price3': lot_df['Price_Model3'],
    'queue': lot_df['QueueLength'],
    'demand': lot_df['NormalizedDemand']
})

# Create Bokeh plot
p = figure(title=f"Real-Time Pricing for Lot: {lot_id}", x_axis_type='datetime', width=950, height=400)
p.line('x', 'price1', source=source, color="gray", legend_label="Model 1", line_width=2)
p.line('x', 'price2', source=source, color="blue", legend_label="Model 2", line_width=2)
p.line('x', 'price3', source=source, color="green", legend_label="Model 3", line_width=2)

p.add_tools(HoverTool(tooltips=[
    ("Time", "@x{%F %H:%M}"),
    ("Model 1", "@price1"),
    ("Model 2", "@price2"),
    ("Model 3", "@price3"),
    ("Queue", "@queue"),
    ("Demand", "@demand")
], formatters={'@x': 'datetime'}))

p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.title.text_font_size = "14pt"

show(p)


# Final Report: Dynamic Pricing for Urban Parking Lots  
**Summer Analytics 2025 Capstone Project**  
Hosted by: Consulting & Analytics Club × Pathway  
Author: S SRISANTOSHI

---

## Submission Guidelines Compliance

This project adheres to all submission requirements listed in the official problem statement.

---

### Google Colab Notebook

- **Well-commented code**
  - Logical blocks for data preprocessing, model implementation, and visualization
  - Clear variable names and explanations for each step

- **All three pricing models included**:
  - **Model 1:** Linear pricing based on occupancy
  - **Model 2:** Demand-based pricing using a custom demand function
  - **Model 3:** Competitive pricing with geospatial awareness and rerouting logic

---

## Demand Function

We used the following formula in Model 2:

\[
\text{Demand} = \alpha \cdot \left( \frac{\text{Occupancy}}{\text{Capacity}} \right) + \beta \cdot \text{QueueLength} - \gamma \cdot \text{TrafficLevel} + \delta \cdot \text{IsSpecialDay} + \varepsilon \cdot \text{VehicleWeight}
\]

- **α** = 1.0 – base demand from occupancy rate  
- **β** = 0.2 – queue length adds demand  
- **γ** = 0.5 – traffic reduces demand  
- **δ** = 0.3 – special events increase demand  
- **ε** = 0.5 – heavier vehicles (e.g., trucks) increase demand  

Demand is **normalized** and applied to the price:
\[
\text{Price} = 10 \cdot (1 + \lambda \cdot \text{NormalizedDemand}), \quad \lambda = 0.8
\]

---

## Assumptions

- Base price: **\$10**  
- Price range: **\$5 to \$20** (bounded for stability)  
- Special day impact = increase in demand  
- Queue length > 4 implies higher urgency  
- Vehicle types are weighted for demand relevance:
  - Truck: 1.5
  - Car: 1.0
  - Bike: 0.7
  - Cycle: 0.5

---

##  How Price Changes with Demand & Competition

- **Model 1:** Price increases proportionally with occupancy
- **Model 2:** Price reflects a weighted demand score based on real-time features
- **Model 3:**  
  - If nearby lots (within 2 km) are cheaper → reroute vehicles and lower price  
  - If nearby lots are more expensive → your price can increase, staying competitive

`SuggestedLot` column identifies where rerouting was applied in Model 3.

---

## Visualizations (Bokeh)

- Line plots showing:
  - Pricing evolution across Model 1, 2, and 3
  - Timestamped pricing behavior for a selected lot
- Hover tooltips with:
  - Queue length  
  - Demand score  
  - Final price  
  - Suggested reroute destination (if any)

---

## Real-Time Simulation with Pathway (Optional Extension)

- Placeholder included to allow streaming with:
  - `pathway.io.csv.read` for time-based streaming
  - `pathway.io.print` for output
- Can be integrated as future work

> Resources: [Pathway Docs](https://pathway.com/developers/user-guide/introduction/first_realtime_app_with_pathway/)

